# Import libraries & parse arguments

In [1]:
import os
import glob
import torch
import numpy as np
import IPython.display
import torch
import torch.nn as nn

from model.embedder import SpeechEmbedder
from datasets.SpeakerVerifyDataset import create_dataset
from utils.hparams import HParam

# Prepare

Get all folder paths (speaker based). Format will be a single list of folder paths

In [2]:
hp = HParam("config.yaml")

/root/voicefilter/utils/hparams.py:18: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  for doc in docs:


In [3]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def test_cuda_collate_fn(batch):
    dvec_list = list()
    target_wav_list = list()
    mixed_wav_list = list()
    target_stft_list = list()
    mixed_stft_list = list()
    mixed_mag_list = list()
    target_mag_list = list()
    mixed_phase_list = list()
    target_phase_list = list()
    
    for _, _, _, dvec_mel, target_wav, mixed_wav, target_mag, target_phase, mixed_mag, mixed_phase, target_stft, mixed_stft, *_ in batch:
        dvec_list.append(dvec_mel)
        target_wav_list.append(target_wav)
        mixed_wav_list.append(mixed_wav)
        target_stft_list.append(target_stft)
        mixed_stft_list.append(mixed_stft)
        mixed_mag_list.append(mixed_mag)
        mixed_phase_list.append(mixed_phase)
        target_mag_list.append(target_mag)
        target_phase_list.append(target_phase)
    target_stft_list = pad_sequence(target_stft_list, batch_first=True)
    mixed_stft_list = pad_sequence(mixed_stft_list, batch_first=True)
    mixed_mag_list = pad_sequence(mixed_mag_list, batch_first=True)
    mixed_phase_list = pad_sequence(mixed_phase_list, batch_first=True)
    target_mag_list = pad_sequence(target_mag_list, batch_first=True)
    target_phase_list = pad_sequence(target_phase_list, batch_first=True)

    return dvec_list, target_mag_list, target_phase_list, \
        mixed_mag_list, mixed_phase_list, \
        target_stft_list, mixed_stft_list, \
        target_wav_list, mixed_wav_list

In [4]:
dataset = create_dataset(hp, ["vin", "zalo-train", "zalo-test"], train=False, size=5000)
# dataloader = DataLoader(dataset=dataset,
#             batch_size=hp.train.batch_size,
#             shuffle=False,
#             num_workers=0,
#             collate_fn=test_cuda_collate_fn,
#             pin_memory=True,
#             drop_last=True,
#             sampler=None)

# Main

In [26]:
embedder_pt = torch.load('embedder.pt',map_location="cpu")
embedder = SpeechEmbedder(hp)
embedder.load_state_dict(embedder_pt)
embedder.eval()

SpeechEmbedder(
  (lstm): LSTM(40, 768, num_layers=3, batch_first=True)
  (proj): LinearNorm(
    (linear_layer): Linear(in_features=768, out_features=256, bias=True)
  )
)

In [50]:
s1_path, s2_path, w1, w2, m1, m2, label = dataset[0]
s1_path, s2_path, w1, w2, m1, m2, label

('datasets/VinBigdata/dataset/50342/50342/vinfast-vsmart-000005356-50342-GiaLai-MienTrung-2001-nam-vinfast-vsmart-1.wav',
 'datasets/VinBigdata/dataset/47530/47530/vinfast-vsmart-000000018-47530-PhuYen-MienTrung-2001-nu-vinfast-vsmart-1.wav',
 array([ 0.00198364,  0.00241089,  0.00131226, ..., -0.00140381,
        -0.00073242, -0.00042725], dtype=float32),
 array([-0.00735474, -0.00769043, -0.0071106 , ...,  0.00170898,
         0.00256348,  0.00247192], dtype=float32),
 tensor([[-4.9013, -4.5955, -5.3387,  ..., -4.0752, -4.4549, -4.4051],
         [-4.1383, -3.4309, -3.4957,  ..., -3.7286, -3.6627, -3.7890],
         [-3.9467, -3.3487, -2.9375,  ..., -3.3519, -2.8895, -3.7211],
         ...,
         [-5.9687, -5.9710, -5.9754,  ..., -5.9916, -5.9917, -5.9955],
         [-5.9884, -5.9813, -5.9787,  ..., -5.9924, -5.9955, -5.9968],
         [-5.9933, -5.9928, -5.9901,  ..., -5.9969, -5.9977, -5.9972]]),
 tensor([[-3.8609, -2.3340, -3.1069,  ..., -3.8614, -4.0769, -3.4283],
         [-2

In [51]:
%%time
with torch.no_grad():
    e1 = embedder(m1)
    e2 = embedder(m2)

CPU times: user 676 ms, sys: 0 ns, total: 676 ms
Wall time: 88.2 ms


In [46]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
cos(e1, e2)

tensor(0.2714)

In [47]:
s1_path, s2_path, w1, w2, m1, m2, label = dataset[1]
s1_path, s2_path, w1, w2, m1, m2, label

('datasets/ZaloAI2020/dataset/778/778-M-32/16k_778-19.wav',
 'datasets/ZaloAI2020/dataset/778/778-M-32/16k_778-15.wav',
 array([0.00527954, 0.00479126, 0.00393677, ..., 0.13638306, 0.12109375,
        0.09823608], dtype=float32),
 array([-0.03295898, -0.04537964, -0.05502319, ...,  0.00564575,
         0.00360107,  0.00280762], dtype=float32),
 tensor([[-4.2474, -4.2779, -3.7135,  ..., -1.5296, -1.4251, -1.4271],
         [-4.5471, -3.4002, -3.5165,  ..., -0.4919, -0.4581, -0.7764],
         [-3.2152, -3.1525, -3.0869,  ..., -0.8979, -0.5293, -0.3553],
         ...,
         [-5.6950, -5.6447, -5.8377,  ..., -4.2674, -4.5451, -4.7328],
         [-5.6883, -5.7253, -5.8072,  ..., -4.7223, -5.1005, -4.9832],
         [-5.7419, -5.6521, -5.7444,  ..., -4.6509, -5.0025, -4.5425]]),
 tensor([[-1.1114, -2.7350, -3.2541,  ..., -4.2496, -4.0953, -3.7908],
         [-0.3636, -1.0051, -1.7304,  ..., -4.0409, -3.0040, -3.2805],
         [-0.5969, -1.0250, -1.8861,  ..., -3.1058, -2.4862, -3.1124],

In [48]:
with torch.no_grad():
    e1 = embedder(m1)
    e2 = embedder(m2)

In [49]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
cos(e1, e2)

tensor(0.5904)